In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import gzip
import numpy as np
import tensorflow as tf
from typing import Tuple

Assuming we achieved pretty good accuracy during the training and testing phases, we can now use the trained model for inference &mdash; in other words, to predict the classification of images that the network has never seen before.

But before we move on, we'll bring in some code you've already seen in previous notebooks. We'll need it later.

In [10]:
labels_map = {
  0: 'T-Shirt',
  1: 'Trouser',
  2: 'Pullover',
  3: 'Dress',
  4: 'Coat',
  5: 'Sandal',
  6: 'Shirt',
  7: 'Sneaker',
  8: 'Bag',
  9: 'Ankle Boot',
}

def read_images(path: str, image_size: int, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  buf = f.read(image_size * image_size * num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
  data = data.reshape(num_items, image_size, image_size)
  return data


def read_labels(path: str, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  f.read(8)
  buf = f.read(num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  data = data.reshape(num_items)
  return data


def get_data(batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
  image_size = 28
  num_train = 60000
  num_test = 10000

  training_images = read_images('data/FashionMNIST/raw/train-images-idx3-ubyte.gz', image_size, num_train)
  test_images = read_images('data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz', image_size, num_test)
  training_labels = read_labels('data/FashionMNIST/raw/train-labels-idx1-ubyte.gz', num_train)
  test_labels = read_labels('data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz', num_test)

  # (training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
  test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  train_dataset = train_dataset.map(lambda image, label: (float(image) / 255.0, label))
  test_dataset = test_dataset.map(lambda image, label: (float(image) / 255.0, label))

  train_dataset = train_dataset.batch(batch_size).shuffle(500)
  test_dataset = test_dataset.batch(batch_size).shuffle(500)

  return (train_dataset, test_dataset)


def get_model() -> tf.keras.Sequential:
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10)
  ])
  return model

Remember that in the previous notebook, after successfully training our network, we saved the weights of the model. We can now reconstruct a trained model using those weights.

In [11]:
model = get_model()
model.load_weights('outputs/weights').expect_partial()

Making a prediction is easy &mdash; we simply call the model's `predict` method and pass it some images. The true power of a neural network is its ability to make predictions for data that has not been seen during training or testing. But for the sake of simplicity, we'll demonstrate how prediction works by re-using a few of the images that we used during testing. The code below retrieves the first three images in our test data and uses them to make predictions. We then print the actual and predicted labels.

In [12]:
batch_size = 64

(_, test_dataset) = get_data(batch_size)
(X_batch, actual_index_batch) = test_dataset.as_numpy_iterator().next()
X = X_batch[0:3, :, :]
actual_indices = actual_index_batch[0:3]

predictions = model.predict(X)

print('\nPredicting:')
for (actual_index, prediction) in zip(actual_indices, predictions):
  actual_name = labels_map[actual_index]
  predicted_index = tf.math.argmax(prediction).numpy()
  predicted_name = labels_map[predicted_index]
  print(f'Actual: {actual_name}, Predicted: {predicted_name}')


Predicting:
Actual: Coat, Predicted: Coat
Actual: Dress, Predicted: Dress
Actual: T-Shirt, Predicted: T-Shirt


With only two epochs of training, our network isn't very accurate, but it should get at least two out of three predictions right. You can increase the number of epochs in the training phase to get better predictions.
